In [11]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [12]:
kwBR=pd.read_csv('SeriesKeywords.csv')
kwMG=pd.read_csv('SeriesKeywordsMg.csv')
conn =sqlite3.connect("indicadores.db")
brasilDf=pd.read_sql_query("select * from brazil_df",conn)
statesDf=pd.read_sql_query("select * from states_df",conn)
conn.close()

In [13]:
#obtendo dados apenas das semanas desejadas dos indicadores
brasilDf=brasilDf.iloc[0:44]
statesDf=statesDf[statesDf['state']=='MG']
statesDf=statesDf.reset_index()
statesDf=statesDf.iloc[0:44]

In [14]:
#reordenando colunas para mais facil visualização
cols=kwMG.columns.tolist()
cols=cols[0:1]+cols[-1:]+cols[1:-1]
kwMG=kwMG[cols]
cols=kwBR.columns.tolist()
cols=cols[0:1]+cols[-1:]+cols[1:-1]
kwBR=kwBR[cols]

In [15]:
#normalizando os dados de indicador a serem usados
brasilDf.loc[:,'new_week_cases']=(brasilDf.loc[:,'new_week_cases']/brasilDf.loc[:,'new_week_cases'].max())*100
brasilDf.loc[:,'new_week_deaths']=(brasilDf.loc[:,'new_week_deaths']/brasilDf.loc[:,'new_week_deaths'].max())*100
statesDf.loc[:,'new_week_cases']=(statesDf.loc[:,'new_week_cases']/statesDf.loc[:,'new_week_cases'].max())*100
statesDf.loc[:,'new_week_deaths']=(statesDf.loc[:,'new_week_deaths']/statesDf.loc[:,'new_week_deaths'].max())*100

,week_last_day,epidemiological_week,total_inhabitants,last_available_confirmed,last_available_deaths,new_week_cases,new_week_deaths,prevalence,mortality,lethality,incidence_cases,incidence_deaths,cases_growth_factor,deaths_growth_factor
0,2020-02-29,9,2.117557e+08,2,0,0.000600,0.000000,0.000944,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000
1,2020-03-07,10,2.117557e+08,19,0,0.005100,0.000000,0.008973,0.000000,0.000000,0.008028,0.000000,8.500000,0.000000
2,2020-03-14,11,2.117557e+08,136,0,0.035097,0.000000,0.064225,0.000000,0.000000,0.055252,0.000000,6.882353,0.000000
3,2020-03-21,12,2.117557e+08,1132,18,0.298777,0.234803,0.534578,0.008500,1.590106,0.470353,0.008500,8.512821,0.000000
4,2020-03-28,13,2.117557e+08,3953,115,0.846235,1.265327,1.866774,0.054308,2.909183,1.332196,0.045808,2.832329,5.388889
5,2020-04-04,14,2.117557e+08,10381,445,1.928252,4.304722,4.902348,0.210148,4.286678,3.035574,0.155840,2.278625,3.402062
6,2020-04-11,15,2.117557e+08,20991,1141,3.182755,9.079050,9.912839,0.538828,5.435663,5.010491,0.328681,1.650591,2.109091
7,2020-04-18,16,2.117557e+08,37175,2375,4.854826,16.097052,17.555608,1.121576,6.388702,7.642770,0.582747,1.525353,1.772989
8,2020-04-25,17,2.117557e+08,59506,4074,6.698784,22.162797,28.101252,1.923915,6.846368,10.545643,0.802340,1.379820,1.376823
9,2020-05-02,18,2.117557e+08,97579,6810,11.421021,35.690060,46.080934,3.215970,6.978961,17.979682,1.292055,1.704939,1.610359


In [16]:
#criando a coluna de semanas nas keywords
semanas=[]
for i in range(len(kwBR)):
    semanas.append(i+9)
kwMG['epidemiological_week']=semanas
kwBR['epidemiological_week']=semanas

In [17]:
#extraindo apenas dados necessários das databases
brasilDf=brasilDf.set_index('epidemiological_week')
statesDf=statesDf.set_index('epidemiological_week')
brasilDf=brasilDf[['new_week_cases','new_week_deaths']]
statesDf=statesDf[['new_week_cases','new_week_deaths']]
kwBR=kwBR.set_index('epidemiological_week')
kwMG=kwMG.set_index('epidemiological_week')
kwBR=kwBR.drop('date',axis=1)
kwMG=kwMG.drop('date',axis=1)

In [136]:
#Função que encontra as maxlag correlations (até 14 semanas de lag pois é 1/3 das semanas)
def getMaxCorrelations(kw,ind,cat):
    dic={
        'Keyword' : [],
        'MaxCorrelation' : [],
        'Lag' : []        
    }
    for i in kw:
        maxcorr=-1
        lag=-1
        for j in range(14):
            corr=kw[:-j].reset_index()[i].corr(ind[j:].reset_index()[cat],method='spearman')
            if corr>=maxcorr:
                maxcorr=corr
                lag=j
        dic['Lag'].append(lag)
        dic['MaxCorrelation'].append(maxcorr)
        dic['Keyword'].append(i)
    return dic

In [139]:
lagcorr=pd.DataFrame(getMaxCorrelations(kwBR,brasilDf,'new_week_cases'))
lagcorr=lagcorr.set_index('Keyword')
lagcorr.sort_values(by='MaxCorrelation',ascending=False)

,MaxCorrelation,Lag
Keyword,,
pcr,0.945470,2
exame covid,0.938506,2
covid pcr,0.927225,2
pcr exame covid,0.921005,3
teste covid,0.875545,3
...,...,...
pcr coronavírus,0.129984,7
tosse,-0.009898,13
álcool gel,-0.021143,13


In [125]:
#função que obtém a correlação a partir de um lag específico
def getSpecificLag(kw,ind,cat,lag=0):
    dic={
        'Keyword' : [],
        'Correlation' : [],
        'Lag' : []
    }
    for i in kw:
        corr=kw[:-lag].reset_index()[i].corr(ind[lag:].reset_index()[cat],method='spearman')
        dic['Correlation'].append(corr)
        dic['Keyword'].append(i)
        dic['Lag'].append(lag)
    return dic

In [127]:
print('Deseja consultar MG ou BR?')
location=input()
print('Qual indicador?(new_week_cases ou new_week_deaths)')
indicator=input()
print('Qual lag?')
lag=int(input())
if(location.upper()=='BR'):
    df=pd.DataFrame(getSpecificLag(kwBR,brasilDf,indicator,lag))
elif(location.upper()=='MG'):
    df=pd.DataFrame(getSpecificLag(kwMG,statesDf,indicator,lag))
df

Deseja consultar MG ou BR?
br
Qual indicador?(new_week_cases ou new_week_deaths)
new_week_cases
Qual lag?
2


,Keyword,Correlation,Lag
0,peguei covid,0.851196,2
1,sintomas do covid,0.522462,2
2,covid-19,-0.073083,2
3,sintomas do covid 19,0.092297,2
4,dor de garganta,-0.392939,2
...,...,...,...
119,respirador,-0.376175,2
120,respirador hospitalar,-0.297302,2
121,medicamento coronavírus,-0.406589,2
122,medicamento corona,-0.487096,2
